# Team A - Lab 2 
### Recommendation Engines

In [55]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [56]:
# # Import excel file for all exercises
data = pd.read_excel('exerciseCB.xlsx', header=1)
# data

In [57]:
df_ITEMS = data.iloc[:20,:11].rename(columns={"Unnamed: 0": "Question"}).set_index('Question')
#df_ITEMS

In [58]:
df_USERPROFILE = data.iloc[25:29,:11].rename(columns={"Unnamed: 0": "User Profile"}).set_index('User Profile')
#df_USERPROFILE

In [59]:
conc = [pd.DataFrame(data.iloc[:20,0]),data.iloc[:20,14:18]]
df_USERS = pd.concat(conc,axis=1).rename(columns={"Unnamed: 0": "Question"}).set_index('Question')
#df_USERS

In [60]:
conc2 = [pd.DataFrame(data.iloc[:20,0]),data.iloc[:20,19:23]]
df_ANSWERS = pd.concat(conc2,axis=1).rename(columns={"Unnamed: 0": "Question"}).set_index('Question')
#df_ANSWERS 

# Simply Unary

In [61]:
## filling the df_USERPROFILE through simply unary

for i in range(len(df_USERPROFILE)):

    u_temp = df_ITEMS.mul(df_USERS.iloc[:,i],axis=0).sum()

    df_USERPROFILE.iloc[i] = u_temp

df_USERPROFILE

,Sports,Books,Leadership,Philosophy,Society,Fiction,Security,Love,VideoGames,Superheroes
User Profile,,,,,,,,,,
User1,3,-2,-1,0,0,2,-1,-1,1,0
User2,-2,2,2,3,-1,-2,0,3,0,-1
User3,-2,1,1,0,0,-3,-1,-2,0,1
User4,0,0,0,0,0,0,0,0,0,0


In [62]:
df_predictions = df_USERS.copy()

df_predictions.iloc[:,:] = None

for i in range(len(df_USERPROFILE)):
    
    user_i = df_ITEMS.mul(df_USERPROFILE.iloc[i,:]).sum(axis=1)
    
    user_i_i = np.sqrt(df_USERPROFILE.iloc[i,:].mul(df_USERPROFILE.iloc[i,:]).sum())
    
    for j in range(len(df_ITEMS)):
        
        quest_j = np.sqrt(df_ITEMS.iloc[j,:].mul(df_ITEMS.iloc[j,:]).sum())
        
        pred_u_i_q_j = user_i.iloc[j] / (user_i_i*quest_j)
        
        df_predictions.iloc[j,i] = pred_u_i_q_j
    

In [63]:
df_predictions = df_predictions.fillna(0)
#df_predictions

In [64]:
total = pd.DataFrame(np.zeros((3,4)),columns=df_predictions.columns,index=['Likes','Dislikes','Neutral'])

total.iloc[0] = (df_predictions>0).sum()
total.iloc[1] = (df_predictions<0).sum()
total.iloc[2] = (df_predictions==0).sum()

total

,User 1,User 2,User 3,User 4
Likes,7.0,15.0,5.0,0.0
Dislikes,11.0,4.0,10.0,0.0
Neutral,2.0,1.0,5.0,20.0


In [65]:
final_rec_LIST = list()

## add one element to the list for recommended questions per user
# we only consider questions the user has not seen yet (neither voted or answered)
# if recommendations nr 5 and 6 tie, display both

for i in range(len(df_predictions.columns)):
    
    final_rec_LIST.append(list(df_predictions
                               .loc[(df_USERS.iloc[:,i].isnull()&
                                     df_ANSWERS.iloc[:,i].isnull()),
                                                  ('User '+ str(i+1))]
                               .nlargest(5,keep='all').index))
    
final_rec = pd.DataFrame(final_rec_LIST).T
final_rec.columns = df_predictions.columns

## taking out any user where there are no predictions available

for i in range(len(df_predictions.columns)):
    
    if all(df_predictions.iloc[:,i]==0) == True:
        
        final_rec.iloc[:,i] = None       

final_rec = final_rec.dropna(how='all')
final_rec

,User 1,User 2,User 3,User 4
0,question12,question14,question14,None
1,question9,question18,question11,None
2,question3,question15,question10,None
3,question18,question7,question18,None
4,question11,question6,question4,None
5,question15,None,None,None


# Unit weight

In [66]:
df_ITEMS_UW = df_ITEMS.divide(df_ITEMS.sum(axis=1),axis=0) # weighing the questions

df_USERPROFILE_UW = df_USERPROFILE.copy()

## filling the df_USERPROFILE_UW through unit weight

for i in range(len(df_USERPROFILE_UW)):

    u_temp = df_ITEMS_UW.mul(df_USERS.iloc[:,i],axis=0).sum()

    df_USERPROFILE_UW.iloc[i] = u_temp

df_USERPROFILE_UW

,Sports,Books,Leadership,Philosophy,Society,Fiction,Security,Love,VideoGames,Superheroes
User Profile,,,,,,,,,,
User1,1.03333,-0.45,-0.25,0.25,0,0.533333,-0.2,-0.25,0.333333,0
User2,-0.533333,0.5,0.55,0.75,-0.2,-0.533333,-0.0833333,0.75,0,-0.2
User3,-0.666667,0.333333,0.25,0,0,-0.916667,-0.333333,-0.75,0,0.0833333
User4,0,0,0,0,0,0,0,0,0,0


In [67]:
df_predictions_UW = df_USERS.copy()

df_predictions_UW.iloc[:,:] = None

for i in range(len(df_USERPROFILE_UW)):
    
    user_i = df_ITEMS_UW.mul(df_USERPROFILE_UW.iloc[i,:]).sum(axis=1)
    
    user_i_i = np.sqrt(df_USERPROFILE_UW.iloc[i,:].mul(df_USERPROFILE_UW.iloc[i,:]).sum())
    
    for j in range(len(df_ITEMS_UW)):
        
        quest_j = np.sqrt(df_ITEMS_UW.iloc[j,:].mul(df_ITEMS_UW.iloc[j,:]).sum())
        
        pred_u_i_q_j = user_i.iloc[j] / (user_i_i*quest_j)
        
        df_predictions_UW.iloc[j,i] = pred_u_i_q_j


In [68]:
df_predictions_UW = df_predictions_UW.fillna(0)
#df_predictions_UW

In [69]:
total_UW = pd.DataFrame(np.zeros((3,4)),columns=df_predictions_UW.columns,index=['Likes','Dislikes','Neutral'])

total_UW.iloc[0] = (df_predictions_UW>0).sum()
total_UW.iloc[1] = (df_predictions_UW<0).sum()
total_UW.iloc[2] = (df_predictions_UW==0).sum()

total_UW

,User 1,User 2,User 3,User 4
Likes,10.0,16.0,4.0,0.0
Dislikes,10.0,4.0,13.0,0.0
Neutral,0.0,0.0,3.0,20.0


In [70]:
final_rec_LIST_UW = list()

## add one element to the list for recommended questions per user
# we only consider questions the user has not seen yet (neither voted or answered)
# if recommendations nr 5 and 6 tie, display both

for i in range(len(df_predictions_UW.columns)):
    
    final_rec_LIST_UW.append(list(df_predictions_UW
                               .loc[(df_USERS.iloc[:,i].isnull()&
                                     df_ANSWERS.iloc[:,i].isnull()),
                                                  ('User '+ str(i+1))]
                               .nlargest(5,keep='all').index))
    
final_rec_UW = pd.DataFrame(final_rec_LIST_UW).T
final_rec_UW.columns = df_predictions_UW.columns

## taking out any user where there are no predictions available

for i in range(len(df_predictions_UW.columns)):
    
    if all(df_predictions_UW.iloc[:,i]==0) == True:
        
        final_rec_UW.iloc[:,i] = None       

final_rec_UW = final_rec_UW.dropna(how='all')
final_rec_UW

,User 1,User 2,User 3,User 4
0,question12,question14,question14,None
1,question9,question18,question11,None
2,question3,question15,question10,None
3,question18,question7,question18,None
4,question15,question11,question4,None


# IDF

In [71]:
# Information Filtering
IDF = data.iloc[21:23,:len(df_USERPROFILE.columns)+1]
IDF.iloc[1,0] = 'IDF'
IDF = IDF.rename(columns={"Unnamed: 0": "Info Filter"}).set_index('Info Filter')

In [72]:
IDF.iloc[0,:] = df_ITEMS.sum()
IDF.iloc[1,:] = np.log10( len(df_ITEMS) / df_ITEMS.sum() )
IDF

,Sports,Books,Leadership,Philosophy,Society,Fiction,Security,Love,VideoGames,Superheroes
Info Filter,,,,,,,,,,
DF,4,6,10,11,6,6,7,6,7,5
IDF,0.69897,0.522879,0.30103,0.259637,0.522879,0.522879,0.455932,0.522879,0.455932,0.60206


In [73]:
# df_ITEMS_UW can be reused

df_USERPROFILE_IDF = df_USERPROFILE_UW.mul(IDF.iloc[1,:])

df_USERPROFILE_IDF

,Sports,Books,Leadership,Philosophy,Society,Fiction,Security,Love,VideoGames,Superheroes
User Profile,,,,,,,,,,
User1,0.722269,-0.235295,-0.0752575,0.0649093,0,0.278869,-0.0911864,-0.13072,0.151977,0
User2,-0.372784,0.261439,0.165566,0.194728,-0.104576,-0.278869,-0.0379943,0.392159,0,-0.120412
User3,-0.46598,0.174293,0.0752575,0,0,-0.479306,-0.151977,-0.392159,0,0.0501717
User4,0,0,0,0,0,0,0,0,0,0


In [74]:
df_predictions_IDF = df_USERS.copy()

df_predictions_IDF.iloc[:,:] = None

for i in range(len(df_USERPROFILE_IDF)):
    
    user_i = df_ITEMS_UW.mul(df_USERPROFILE_IDF.iloc[i,:]).sum(axis=1)
    
    user_i_i = np.sqrt(df_USERPROFILE_IDF.iloc[i,:].mul(df_USERPROFILE_IDF.iloc[i,:]).sum())
    
    for j in range(len(df_predictions_IDF)):
        
        quest_j = np.sqrt(df_ITEMS_UW.iloc[j,:].mul(df_ITEMS_UW.iloc[j,:]).sum())
        
        pred_u_i_q_j = user_i.iloc[j] / (user_i_i*quest_j)
        
        df_predictions_IDF.iloc[j,i] = pred_u_i_q_j


In [75]:
df_predictions_IDF = df_predictions_IDF.fillna(0)
#df_predictions_IDF

In [76]:
total_IDF = pd.DataFrame(np.zeros((3,4)),columns=df_predictions_IDF.columns,index=['Likes','Dislikes','Neutral'])

total_IDF.iloc[0] = (df_predictions_IDF>0).sum()
total_IDF.iloc[1] = (df_predictions_IDF<0).sum()
total_IDF.iloc[2] = (df_predictions_IDF==0).sum()

total_IDF

,User 1,User 2,User 3,User 4
Likes,10.0,14.0,5.0,0.0
Dislikes,10.0,6.0,14.0,0.0
Neutral,0.0,0.0,1.0,20.0


In [77]:
final_rec_LIST_IDF = list()

## add one element to the list for recommended questions per user
# we only consider questions the user has not seen yet (neither voted or answered)
# if recommendations nr 5 and 6 tie, display both

for i in range(len(df_predictions_IDF.columns)):
    
    final_rec_LIST_IDF.append(list(df_predictions_IDF
                               .loc[(df_USERS.iloc[:,i].isnull()&
                                     df_ANSWERS.iloc[:,i].isnull()),
                                                  ('User '+ str(i+1))]
                               .nlargest(5,keep='all').index))
    
final_rec_IDF = pd.DataFrame(final_rec_LIST_IDF).T
final_rec_IDF.columns = df_predictions_IDF.columns

## taking out any user where there are no predictions available

for i in range(len(df_predictions_IDF.columns)):
    
    if all(df_predictions_IDF.iloc[:,i]==0) == True:
        
        final_rec_IDF.iloc[:,i] = None       

final_rec_IDF = final_rec_IDF.dropna(how='all')
final_rec_IDF

,User 1,User 2,User 3,User 4
0,question12,question14,question14,None
1,question9,question7,question11,None
2,question3,question18,question10,None
3,question18,question15,question18,None
4,question15,question10,question4,None


# Switched Hybrid

### User 4 cold-start problem

Switching the content-based to **non-personalise for users without actions** collected.

In [78]:
# For our switched hybrid, we propose a mix of the below 4 methods to obtain the predictions for the non-personalized part

## The personalized engine will follow the IDF setup.

## The non-personalized engine may take three different approaches, we recommend #3

# 1 - take mean of all predictions

# 2 - likes matrix: more likes than dislikes

### 3 - answer matrix: assign weights on overall popularity of a question;
#
# find a measure for traffic on questions, i.e. its popularity; 
# as a proxy, we are defining popularity as user interaction = the sum of absolute up & down votes of answers

### LIMITATION: would rather have total number of up & down votes, not the aggregate of it;
#               however, taking the absolute from the data provided gives us the closest approximation
#               of total popularity of a specific question
####

# 4 - additional info required: answer matrix enhanced with one more column to show both # of up- and down-votes


switch = 'approach_3' # chose which approach to fill the cold-start users' predictions


if switch == 'approach_1':
    
    SwitchedHybrid = df_predictions_IDF.mean(axis=1)

elif switch == 'approach_2':
    
    agg_likes = df_USERS.sum(axis=1)
    sum_scale = agg_likes.abs().sum()
    
    SwitchedHybrid = agg_likes/sum_scale

elif switch == 'approach_3':
    
    approach3 = df_ANSWERS.abs().sum(axis=1)
    
    SwitchedHybrid = approach3 / approach3.sum() # scaling the predictions

#else: ## approach 4 - if data available

    

In [79]:
df_predictions_SH2 = df_predictions_IDF.copy()
final_rec_SH2 = final_rec_IDF.copy()

## filling the recommendation matrix for all the users where no ratings/predictions are available

for i in range(len(df_predictions_IDF.columns)):
    
    if all(df_predictions_IDF.iloc[:,i]==0) == False:
        
        continue
        
    else:
        
        df_predictions_SH2.iloc[:,i] = SwitchedHybrid # for sake of completeness, store in prediction matrix
        
        final_rec_SH2.iloc[:,i] = SwitchedHybrid.nlargest(5,keep='all').index


final_rec_SH2

,User 1,User 2,User 3,User 4
0,question12,question14,question14,question19
1,question9,question7,question11,question17
2,question3,question18,question10,question20
3,question18,question15,question18,question2
4,question15,question10,question4,question16


# Hybrid Challenge

For the hybrid challenge, please find our proposed Recommendation System in a separate PDF.

### END OF DOCUMENT